In [67]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta

In [68]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\data'
asset_data = 'AssetData.csv'
asset_system = 'AssetStructureA&S.csv'
system_component = 'AssetStructureS&C.csv'
component_dutie = 'DutieStructureC&D.csv'
dutie_data = 'DutieStructureD&D.csv'
dutie_resource = 'DutieStructureD&R.csv'
wbs = 'WBS.csv'

ruteA_D = os.path.join(folder,asset_data)
ruteA_S = os.path.join(folder,asset_system)
ruteS_C = os.path.join(folder,system_component)
ruteC_D = os.path.join(folder,component_dutie)
ruteD_D = os.path.join(folder,dutie_data)
ruteD_R = os.path.join(folder,dutie_resource)
rute_wbs = os.path.join(folder,wbs)


In [69]:
dataA_D = pd.read_csv(ruteA_D, dtype = {'Cod sap':str})
dataA_S = pd.read_csv(ruteA_S, dtype = {'System':str})
dataS_C = pd.read_csv(ruteS_C)
dataC_D = pd.read_csv(ruteC_D)
dataD_D = pd.read_csv(ruteD_D, dtype = {'Specialist':str,'Amount Craft':int,'Time':float})
dataD_R = pd.read_csv(ruteD_R, dtype = {'Cod. Material':str})
data_wbs = pd.read_csv(rute_wbs)
dataD_D.loc[:,'Last date task done']=pd.to_datetime(dataD_D.loc[:,'Last date task done'],format='%d/%m/%Y')

In [70]:
# DATA OF ASSET INFORMATION
# Removing Spanish information
asset_SP_EN = ['WBS_1_ES', 'WBS_2_ES', 'WBS_3_ES', 'WBS_4_ES']
dataA_D_EN = dataA_D.drop(asset_SP_EN, axis=1)
# All Johan&Jorge clearence
dataA_D_JJ = dataA_D_EN.loc[dataA_D.loc[:,'Responsable']=='Johan&Jorge',:].sort_values(by='Asset tag').reset_index(drop = True)
dataA_D_JJ.to_excel('dataAD.xlsx',encoding='utf-8-sig',index=False)

In [71]:
# Getting System data with out other reponsable than Johan and Jorge
dataA_S_A = dataA_S
dataA_S_JJ = pd.merge(dataA_S_A,dataA_D_EN.loc[:,['Asset tag','Responsable']], how = 'left',left_on='Asset tag',right_on='Asset tag')
dataA_S_JJ = dataA_S_JJ.loc[dataA_S_JJ.loc[:,'Responsable'] == 'Johan&Jorge',:].reset_index(drop=True).sort_values(by='key_system')
dataA_S_JJ.drop(columns={'Responsable'},inplace = True)
dataA_S_JJ.to_excel('dataAS.xlsx',encoding='utf-8-sig',index=True)

In [72]:
# Getting Components data with out other reponsable than Johan and Jorge
dataS_C_A = dataS_C
dataS_C_A.loc[:,'key_asset'] = dataS_C.loc[:,'key_system'].str.split("|").str[0]
dataS_C_JJ = pd.merge(dataS_C_A,dataA_D_EN.loc[:,['Asset tag','Responsable']], how = 'left',left_on='key_asset',right_on='Asset tag')
dataS_C_JJ = dataS_C_JJ.loc[dataS_C_JJ.loc[:,'Responsable'] == 'Johan&Jorge',:].reset_index(drop=True).sort_values(by='key_component')
dataS_C_JJ.drop(columns={'key_asset','Asset tag', 'Responsable'},inplace = True)
dataS_C_JJ.to_excel('dataSC.xlsx',encoding='utf-8-sig',index=True)

In [73]:
# Getting Duties data with out other reponsable than Johan and Jorge
dataC_D_A = dataC_D
dataC_D_A.loc[:,'key_asset'] = dataC_D.loc[:,'key_component'].str.split("|").str[0]
dataC_D_JJ = pd.merge(dataC_D_A,dataA_D_EN.loc[:,['Asset tag','Responsable']], how = 'left',left_on='key_asset',right_on='Asset tag')
dataC_D_JJ = dataC_D_JJ.loc[dataC_D_JJ.loc[:,'Responsable'] == 'Johan&Jorge',:].reset_index(drop=True).sort_values(by='key_dutie')
dataC_D_JJ.drop(columns={'key_asset','Asset tag', 'Responsable'},inplace = True)
dataC_D_JJ.to_excel('dataCD.xlsx',encoding='utf-8-sig',index=True)

In [84]:
## DATA OF DUTIES AND TASK INFORMATION
# Getting Duties data with out other reponsable than Johan and Jorge
dataD_D_A = dataD_D
dataD_D_A.loc[:,'key_asset'] = dataD_D.loc[:,'key_dutie'].str.split("|").str[0]
dataD_D_JJ = pd.merge(dataD_D_A,dataA_D_EN.loc[:,['Asset tag','Responsable']], how = 'left',left_on='key_asset',right_on='Asset tag')
dataD_D_JJ = dataD_D_JJ.loc[dataD_D_JJ.loc[:,'Responsable'] == 'Johan&Jorge',:].reset_index(drop=True)
dataD_D_JJ.drop(columns={'key_asset','Asset tag', 'Responsable'},inplace = True)

# Keeping just Mechanical and Predective specialist
dataD_D_MP = dataD_D_JJ.loc[(dataD_D_JJ.loc[:,'Specialist']=='TEC-MEC') | (dataD_D_JJ.loc[:,'Specialist']=='TEC-PRD'),:].reset_index(drop = True)
dataD_D_M = dataD_D_JJ.loc[dataD_D_JJ.loc[:,'Specialist']=='TEC-MEC',:].reset_index(drop = True).sort_values(by='key_dutie')
dataD_D_P = dataD_D_JJ.loc[dataD_D_JJ.loc[:,'Specialist']=='TEC-PRD',:].reset_index(drop = True).sort_values(by='key_dutie')

# Loading Duties data
dataD_D_M.to_excel('dataDD_M.xlsx',encoding = 'utf-8-sig',index=False)
dataD_D_P.to_excel('dataDD_P.xlsx',encoding = 'utf-8-sig',index=False)

# All the non Mechanical or Predective activities are remove
dataD_D_M.loc[:,'Man hour'] = dataD_D_M.loc[:,'Time']*dataD_D_M.loc[:,'Amount Craft']
dataD_D_P.loc[:,'Man hour'] = dataD_D_P.loc[:,'Time']*dataD_D_P.loc[:,'Amount Craft']

rowfilterM = ~dataD_D_M.loc[:,'Frequency'].isnull()
rowfilterP = ~dataD_D_P.loc[:,'Frequency'].isnull()

dataD_D_M.loc[rowfilterM,'Strategy'] = dataD_D_M.loc[rowfilterM,'Frequency'].astype(int).astype(str) + dataD_D_M.loc[rowfilterM,'Unidad Periodo']

dataD_D_P.loc[rowfilterP,'Strategy'] = dataD_D_P.loc[rowfilterP,'Frequency'].astype(int).astype(str) + dataD_D_P.loc[rowfilterP,'Unidad Periodo']

# All the conditional activities are remove (1T: first task: all activities with frequency 2T second task: all conditional activities)
dataD_D_M_1T = dataD_D_M.loc[dataD_D_M.loc[:,'Maintenance type'] != 'COND',:].reset_index(drop = True)
dataD_D_P_1T = dataD_D_P.loc[dataD_D_P.loc[:,'Maintenance type'] != 'COND',:].reset_index(drop = True)
dataD_D_MP_2T = dataD_D_MP.loc[dataD_D_MP.loc[:,'Maintenance type'] == 'COND',:].reset_index(drop = True)

In [74]:
%%time
# Merging all datas csv 
# Asset&Data(Johan&Jorge) -> Asset&System -> System&Component -> Component&Duties -> 
# Duties&Data(Mechanical/Predective/Mechanical&Predective)
dataA_D_S = pd.merge(dataA_D_JJ,dataA_S_JJ,how = 'left',on = 'Asset tag')
dataA_S_C = pd.merge(dataA_D_S, dataS_C_JJ, how='left', on = 'key_system')
dataA_S_C_D = pd.merge(dataA_S_C, dataC_D_JJ, how='left', on='key_component')
# Merging all with conditional activities
dataASCD_M = pd.merge(dataA_S_C_D, dataD_D_M, how = 'inner', on = 'key_dutie')
dataASCD_P = pd.merge(dataA_S_C_D, dataD_D_P, how = 'inner', on = 'key_dutie')
# Merging all without conditional activities, just first task
dataASCD_1TM = pd.merge(dataA_S_C_D, dataD_D_M_1T, how = 'inner', on = 'key_dutie')
dataASCD_1TP = pd.merge(dataA_S_C_D, dataD_D_P_1T, how = 'inner', on = 'key_dutie')
# Saving the data
#dataASCD_1TM.to_excel('dataASCD_1TM.xlsx',encoding='utf-8-sig', index=False)
#dataASCD_1TP.to_excel('dataASCD_1TP.xlsx',encoding='utf-8-sig', index=False)

#dataASCD_M.to_excel('dataASCD_M.xlsx',encoding='utf-8-sig', index=False)
#dataASCD_P.to_excel('dataASCD_P.xlsx',encoding='utf-8-sig', index=False)

Wall time: 50.9 ms


In [75]:
# Adding the pivot column to project
dataASCD_1TM.loc[:,'pivot'] = dataASCD_1TM.loc[:,'Asset tag'] + '-' + dataASCD_1TM.loc[:,'Strategy'] 
dataASCD_1TP.loc[:,'pivot'] = dataASCD_1TP.loc[:,'Asset tag'] + '-' + dataASCD_1TP.loc[:,'Strategy'] 

In [76]:
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo','Strategy', 'Date task done']

In [77]:
projection = dataASCD_1TM.loc[:,['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']].groupby(['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']).size().reset_index()
projection.drop(columns = {0},inplace =True)
projection.loc[:,'Date plan'] = projection.loc[:,'Last date task done']
[row,column] = projection.shape

In [78]:
%%time
planDates = pd.DataFrame(columns = projection.columns)
projectionflag = projection

for m in range(row):
    while projectionflag.loc[m,'Date plan'] < date(2022,1,1):
        hour2days = round(projectionflag.loc[m,'Frequency']/projectionflag.loc[m,'Hour day'],0)
        if projectionflag.loc[m,'Unidad Periodo'] == 'H':
            delta=timedelta(days = hour2days)
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'S':
            delta = timedelta(weeks = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'M':
            delta = timedelta(weeks = 4*projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'A':
            delta = timedelta(weeks = 52*projectionflag.loc[m,'Frequency'])
        
        projectionflag.loc[m,'Date plan'] = projectionflag.loc[m,'Date plan'] + delta
        planDates = planDates.append(projectionflag.loc[m,:],ignore_index=True)
planDates.loc[:,'pivot'] = planDates.loc[:,'Asset tag'] + '-' + planDates.loc[:,'Strategy']

Wall time: 24.6 s


In [79]:
planDates

,Asset tag,Strategy,Frequency,Unidad Periodo,Last date task done,Hour day,Date plan,pivot
0,2221-PU-201,1000H,1000.0,H,2021-03-25,24,2021-05-06,2221-PU-201-1000H
1,2221-PU-201,1000H,1000.0,H,2021-03-25,24,2021-06-17,2221-PU-201-1000H
2,2221-PU-201,1000H,1000.0,H,2021-03-25,24,2021-07-29,2221-PU-201-1000H
3,2221-PU-201,1000H,1000.0,H,2021-03-25,24,2021-09-09,2221-PU-201-1000H
4,2221-PU-201,1000H,1000.0,H,2021-03-25,24,2021-10-21,2221-PU-201-1000H
...,...,...,...,...,...,...,...,...
6521,4561-TK-40888,1M,1.0,M,2021-03-25,24,2021-10-07,4561-TK-40888-1M
6522,4561-TK-40888,1M,1.0,M,2021-03-25,24,2021-11-04,4561-TK-40888-1M
6523,4561-TK-40888,1M,1.0,M,2021-03-25,24,2021-12-02,4561-TK-40888-1M
6524,4561-TK-40888,1M,1.0,M,2021-03-25,24,2021-12-30,4561-TK-40888-1M


In [80]:
%%time
dataASCD_1TMPlan = pd.merge(dataASCD_1TM,planDates.loc[:,['pivot','Date plan']],how='left',on='pivot')
# save the datacompileMplan
#dataASCD_1TMPlan.to_excel('dataASCD_1TMPlan.xlsx', encoding='utf-8-sig')

Wall time: 49.9 ms


In [83]:
dataASCD_1TMPlan.shape

(30257, 30)

In [ ]:
# DATA OF DATES AND PLANIFICATION
dataStrategy = dataAllcompileM.loc[:,['Frequency','Unidad Periodo']].groupby(['Frequency','Unidad Periodo']).size().reset_index()
dataStrategy.loc[:,0] = dataStrategy.loc[:,'Frequency'].astype(int).astype(str) + dataStrategy.loc[:,'Unidad Periodo']
dataStrategy.rename(columns = {0:'claveStrategy'},inplace =True)

In [ ]:
# Data wrangling
filtertype = dataAllcompileM.loc[:,'Maintenance type'] == 'INSP'
filterunidad = dataAllcompileM.loc[:,'Unidad Periodo'] == 'H'
filterhour = dataAllcompileM.loc[:,'Frequency'] == 1400
dataAllcompileM.loc[filtertype & filterunidad,'Frequency_eq'] = (dataAllcompileM.loc[filtertype & filterunidad,'Frequency']/(24*7)).round(0)

In [ ]:
dataAllcompileM.loc[filtertype & filterunidad,['Frequency','Frequency_eq']].groupby(['Frequency','Frequency_eq']).size()

In [ ]:
# Combine activities based on similar columns
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo']
# '2253-PU-212'
rowfilter = dataAllcompileM.loc[:,'Asset tag'] == '2221-PU-201'
# dataAllcompile.loc[rowfilter,columnfilter]
datacombine = dataAllcompileM.loc[rowfilter,columnfilter].groupby(['key_system','Component','Maintenance type','Constraint','Frequency']).agg({'Primary task': lambda x: "\n".join(x)}).reset_index()

In [ ]:
datacombine